<a href="https://colab.research.google.com/github/luizhsalazar/LLM/blob/main/rag_llama_index_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama-Index with Pinecone

## [Video covering this notebook](https://youtu.be/4kwAhzzaW4A)

In this notebook, we will demo how to use the `llama-index` (previously GPT-index) library with Pinecone for semantic search.

In [ ]:
%%capture
!pip install llama-index datasets pinecone-client openai transformers

# Using langchain
# !pip install -qU \
#     langchain==0.0.292 \
#     openai==0.28.0 \
#     tiktoken==0.5.1

# Teste

We can go ahead and load the [SQuAD dataset from Huggingface](https://huggingface.co/datasets/squad), which contains questions and answer pairs from Wikipedia articles.

[Datasets](https://github.com/huggingface/datasets) github repo.

We'll then convert the dataset into a pandas DataFrame and keep only the unique 'context' fields, which are the text passages that the questions are based on.

In [ ]:
from datasets import load_dataset

data = load_dataset('squad', split='train')
data = data.to_pandas()[['id', 'context', 'title']]
data.drop_duplicates(subset='context', keep='first', inplace=True)
data.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

,id,context,title
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",University_of_Notre_Dame
5,5733bf84d058e614000b61be,"As at most other universities, Notre Dame's st...",University_of_Notre_Dame
10,5733bed24776f41900661188,The university is the major seat of the Congre...,University_of_Notre_Dame
15,5733a6424776f41900660f51,The College of Engineering was established in ...,University_of_Notre_Dame
20,5733a70c4776f41900660f64,All of Notre Dame's undergraduate students are...,University_of_Notre_Dame


In [ ]:
data.shape

(18891, 3)

The following code transforms our DataFrame into a list of Document objects, ready for indexing with llama_index.

Each document contains the follwing:
- text passage
- a unique id
- an extra field for the article title.

In [ ]:
from llama_index import Document

docs = []

for i, row in data.iterrows():
    docs.append(Document(
        text=row['context'],
        doc_id=row['id'],
        extra_info={'title': row['title']}
    ))
print(len(docs))
docs[0]

18891


Document(id_='5733be284776f41900661182', embedding=None, metadata={'title': 'University_of_Notre_Dame'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6', text='Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', start_char_idx=None, end_char_idx=N

In [ ]:
documents = docs[:10]
len(documents)

10

### Indexing in Pinecone

[Pinecone](https://www.pinecone.io/) is a managed vector database service designed for machine learning applications. We're using it in this context to store and retrieve embeddings generated by our language model, enabling efficient and scalable semantic similarity-based search.

Get the relevant API key and environment that we [get for **free** in the console](https://app.pinecone.io/), then create a new index.

The index has a dimension of 1536 and uses cosine similarity, which is the recommended metric for comparing vectors produced by the `text-embedding-ada-002` model we'll be using.

In [ ]:
from google.colab import userdata

'51a79d37-716e-40ec-9cf7-c0f9d6dd1dca'

In [ ]:
import pinecone
import os

# find API key in console at app.pinecone.io
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
# environment is found next to API key in the console
os.environ['PINECONE_ENVIRONMENT'] = userdata.get('PINECONE_ENVIRONMENT')

# initialize connection to pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENVIRONMENT']
)

# create the index if it does not exist already
index_name = 'llama-index-pinecone'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )

# connect to the index
pinecone_index = pinecone.Index(index_name)

Here, we're initializing a `PineconeVectorStore` with our previously created Pinecone index. This object will serve as the storage and retrieval interface for our document embeddings in Pinecone's vector database.

In [ ]:
from llama_index.vector_stores import PineconeVectorStore

# we can select a namespace (acts as a partition in an index)
namespace = '' # default namespace

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

Next we initialize the `GPTVectorStoreIndex` with our list of `Document` objects, using the `PineconeVectorStore` as storage and `OpenAIEmbedding` model for embeddings.

`StorageContext` is used to configure the storage setup, and `ServiceContext` sets up the embedding model. The `GPTVectorStoreIndex` handles the indexing and querying process, making use of the provided storage and service contexts.

## We used the default one, How to Customize It ??


In [ ]:
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms import OpenAI
# alternatively
# from langchain.llms import ...
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")


#define LLM
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo", max_tokens=512)

# configure service context
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=500, chunk_overlap=50)

# set_global_service_context(service_context)
index = VectorStoreIndex.from_documents(documents, service_context=service_context)



In [ ]:
from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding

# setup our storage (vector db)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

import os
# https://platform.openai.com/account/api-keys
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"


# setup the index/query process, ie the embedding model (and completion if used)
embed_model = OpenAIEmbedding(model='text-embedding-ada-002', embed_batch_size=100)
service_context = ServiceContext.from_defaults(embed_model=embed_model)

index = GPTVectorStoreIndex.from_documents(
    documents, storage_context=storage_context,
    service_context=service_context
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Upserted vectors:   0%|          | 0/100 [00:00<?, ?it/s]

Finally we can build a query engine from the `index` we build and use this engine to perform a query.

In [ ]:
index??

In [ ]:
query_engine = index.as_query_engine()
res = query_engine.query("In what year was the college of engineering established at the University of Notre Dame?")
print(res)

The college of engineering was established in 1920 at the University of Notre Dame.


In [ ]:
data.context[20]

"All of Notre Dame's undergraduate students are a part of one of the five undergraduate colleges at the school or are in the First Year of Studies program. The First Year of Studies program was established in 1962 to guide incoming freshmen in their first year at the school before they have declared a major. Each student is given an academic advisor from the program who helps them to choose classes that give them exposure to any major in which they are interested. The program also includes a Learning Resource Center which provides time management, collaborative learning, and subject tutoring. This program has been recognized previously, by U.S. News & World Report, as outstanding."

In [ ]:
query_engine = index.as_query_engine()
res = query_engine.query("When was the First Year of Studies program established at the University of Notre Dame has?")
print(res)

The First Year of Studies program was established in 1962 at the University of Notre Dame.


Delete the `index` if not necessary.

In [ ]:
pinecone.delete_index(index_name)

If you want to learn more into pinecone, you can visit the [pinecone github examples](https://github.com/pinecone-io/examples/tree/master)

---